`In progress`

# training.ipynb

Training implementation.

Author: Connacher Murphy

In [23]:
# Libraries
import pest_classification as pest

import os
from sklearn.metrics import accuracy_score
import timm
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from tqdm import tqdm
from types import SimpleNamespace

In [24]:
# Split df into test and training
# CM: sampling to speed up execution
df = pest.df
train_df = df[df["set"] == "train_set"].sample(256)

In [25]:
# Dataloader
config = SimpleNamespace(**{})

config.batch_size = 32

config.image_dir = os.path.expanduser("~/data/ccmt/CCMT Dataset-Augmented")
config.image_size = 256

train_dataset = pest.AugmentedCCMT(config, train_df)

train_dataloader = DataLoader(
    train_dataset,
    batch_size=config.batch_size,
    shuffle=True,
    num_workers=0,
)
# test_dataloader = DataLoader(
#     test_dataset,
#     batch_size=config.batch_size,
#     shuffle=True,
#     num_workers=0,
# )


In [26]:
# config.num_classes = len(pest.crop_classes["Maize"])
config.num_classes = 2
config.backbone = "resnet18"

model = timm.create_model(config.backbone, pretrained=True, num_classes=config.num_classes)

In [27]:
X = torch.randn(config.batch_size, 3, config.image_size, config.image_size)
y = model(X)
print(y.shape)
print(y)

torch.Size([32, 2])
tensor([[ 0.0294,  0.0772],
        [ 0.0350, -0.0439],
        [ 0.0515,  0.1350],
        [-0.0292,  0.0332],
        [ 0.0566,  0.1282],
        [-0.0017,  0.0740],
        [-0.0352,  0.0528],
        [-0.0035,  0.0223],
        [-0.0097,  0.0047],
        [-0.0349, -0.0318],
        [-0.0222, -0.0822],
        [-0.0032, -0.0365],
        [-0.0122, -0.0670],
        [-0.0273,  0.1073],
        [-0.1124, -0.0081],
        [-0.1228,  0.1702],
        [-0.0266, -0.0468],
        [-0.0237, -0.0735],
        [-0.0089, -0.0211],
        [ 0.0149,  0.0330],
        [-0.0027, -0.0157],
        [-0.0378, -0.1109],
        [-0.0202, -0.0607],
        [-0.1529, -0.0546],
        [ 0.0069, -0.0062],
        [-0.0643,  0.0015],
        [-0.0321, -0.0116],
        [-0.0165, -0.0272],
        [ 0.0037, -0.0613],
        [-0.0067, -0.0320],
        [-0.0160,  0.0566],
        [ 0.1356,  0.0848]], grad_fn=<AddmmBackward0>)


In [28]:
config.criterion = nn.CrossEntropyLoss()

In [29]:
config.lr = 1e-4

optimizer = torch.optim.Adam(model.parameters(), lr=config.lr, weight_decay=0.0)

In [30]:
print(torch.cuda.is_available())
config.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

False


In [31]:
model.to(config.device)

def calculate_metric(y, y_pred):
  metric = accuracy_score(y, y_pred)
  return metric

def train(dataloader, model, optimizer, config):
    for epoch in range(config.num_epochs):
        print(f"Epoch {epoch + 1}")
        epoch_loss = pest.train_epoch(dataloader, model, optimizer, config)

        # Calculate accuracy
        for i, (images, labels) in enumerate(dataloader):
            images = images.to(config.device)
            labels = labels.to(config.device)

            # Forward pass
            outputs = model(images)
            labels_pred = torch.argmax(outputs, dim=1)
            
            accuracy = accuracy_score(labels, labels_pred)
            
            print(f"Batch {i} accuracy ={accuracy}")
            

        print(f"Epoch loss = {epoch_loss}")

In [32]:
config.num_epochs = 3

In [33]:
train(train_dataloader, model, optimizer, config)

Epoch 1


100%|██████████| 8/8 [00:24<00:00,  3.09s/it]


Batch 0 accuracy =0.96875
Batch 1 accuracy =0.9375
Batch 2 accuracy =0.9375
Batch 3 accuracy =0.84375
Batch 4 accuracy =0.875
Batch 5 accuracy =0.875
Batch 6 accuracy =0.875
Batch 7 accuracy =0.875
Epoch loss = 0.677024245262146
Epoch 2


100%|██████████| 8/8 [00:24<00:00,  3.09s/it]


Batch 0 accuracy =1.0
Batch 1 accuracy =1.0
Batch 2 accuracy =1.0
Batch 3 accuracy =1.0
Batch 4 accuracy =1.0
Batch 5 accuracy =1.0
Batch 6 accuracy =1.0
Batch 7 accuracy =1.0
Epoch loss = 0.5539707988500595
Epoch 3


100%|██████████| 8/8 [00:24<00:00,  3.07s/it]


Batch 0 accuracy =1.0
Batch 1 accuracy =1.0
Batch 2 accuracy =1.0
Batch 3 accuracy =1.0
Batch 4 accuracy =1.0
Batch 5 accuracy =1.0
Batch 6 accuracy =1.0
Batch 7 accuracy =1.0
Epoch loss = 0.4574759565293789
